In [4]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from transformers import ViTForImageClassification
from transformers import AutoFeatureExtractor

ModuleNotFoundError: No module named 'transformers'

In [5]:
!pip3 install transformers

  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/0f/12/d8e27a190ca67811f81deea3183b528d9169f10b74d827e0b9211520ecfa/transformers-4.36.0-py3-none-any.whl.metadata
  Using cached transformers-4.36.0-py3-none-any.whl.metadata (126 kB)
  Obtaining dependency information for huggingface-hub<1.0,>=0.19.3 from https://files.pythonhosted.org/packages/05/09/1945ca6ba3ad8ad6e2872ba682ce8d68c5e63c8e55458ed8ab4885709f1d/huggingface_hub-0.19.4-py3-none-any.whl.metadata
  Using cached huggingface_hub-0.19.4-py3-none-any.whl.metadata (14 kB)
  Obtaining dependency information for regex!=2019.12.17 from https://files.pythonhosted.org/packages/d1/f1/9c50c0e1e76234f05f876dd49df925dae49da7fb8cb152a429006c71f65b/regex-2023.10.3-cp310-cp310-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 419.0 kB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for tokenizers<0.19,>=0.14 from https://files.pythonhosted.org/pac

In [2]:
!huggingface-cli login

In [3]:

class MedicalImagesDataset(datasets.ImageFolder):
    def _init_(self, root_dir, transform=None):
        super(MedicalImagesDataset, self)._init_(root=root_dir, transform=transform)

transformations = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.ToTensor(),
])


train_dir = '../data/train'  
val_dir = '../data/test'  

train_dataset = MedicalImagesDataset(train_dir, transform=transformations)
val_dataset = MedicalImagesDataset(val_dir, transform=transformations)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# ViT model
feature_extractor = AutoFeatureExtractor.from_pretrained('google/vit-huge-patch14')
model = ViTForImageClassification.from_pretrained('google/vit-huge-patch14', num_labels=len(train_dataset.classes))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-5)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

FileNotFoundError: [Errno 2] No such file or directory: '../../data/train'

In [ ]:
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images).logits
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader)}')

    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images).logits
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print(f'Validation Accuracy: {100 * correct / total}%')

print("Training complete")